In [1]:
from datetime import datetime, timedelta
from tqdm import tqdm
from ray.util.client import ray

from run_simulation import run_simulations, metrics

ray.init(_memory=16 * 1024 ** 3, num_cpus=8, num_gpus=0, include_dashboard=True, ignore_reinit_error=True)

2024-10-28 15:40:35,682	INFO worker.py:1743 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2024-10-28 15:40:38,511	INFO logservicer.py:103 -- New logs connection established. Total clients: 1


Python version:,3.9.18
Ray version:,2.10.0
Dashboard:,http://127.0.0.1:8265


In [2]:
from gymportal.data.ev_generators import get_standard_generator, RealWorldGenerator
from acnportal.acnsim import Linear2StageBattery
from gymportal.data.battery_generators import CustomizableBatteryGenerator
from gymportal.sim import get_charging_network, Recomputer, EvaluationSimulator, SimGenerator
from icecream import ic

charging_network = get_charging_network('simple_acn', basic_evse=True, voltage=208,
                                        network_kwargs={
                                            'station_ids': ['CA-504', 'CA-503', 'CA-502', 'CA-501'],
                                             #'station_ids': ['CA-501'],
"aggregate_cap": 32 * 208 / 1000})

# charging_network = get_charging_network('caltech', basic_evse=True, voltage=208,
#                                         network_kwargs={"transformer_cap": 40 * 32 * 208 / 1000})

battery_generator = CustomizableBatteryGenerator(voltage=208,
                                                 period=1,
                                                 battery_types=[
                                                     Linear2StageBattery],
                                                 max_power_function='normal')

ev_generator = RealWorldGenerator(battery_generator=battery_generator, site='caltech', period=1)
# ev_generator = get_standard_generator('caltech', battery_generator, seed=42)

train_generator = SimGenerator(
    charging_network=charging_network,
    simulation_days=7,
    n_intervals=46,
    start_date=datetime(2019, 1, 1),
    ev_generator=ev_generator,
    recomputer=Recomputer(recompute_interval=10, sparse=True),
    sim_class=EvaluationSimulator,
)

ic(train_generator.end_date + timedelta(days=1))

eval_generator = SimGenerator(
    charging_network=charging_network,
    simulation_days=7,
    n_intervals=1,
    start_date=train_generator.end_date + timedelta(days=1),
    ev_generator=ev_generator,
    recomputer=Recomputer(recompute_interval=10, sparse=True),
    sim_class=EvaluationSimulator,
)

ic(eval_generator.end_date + timedelta(days=1))

validation_generator = SimGenerator(
    charging_network=charging_network,
    simulation_days=14,
    n_intervals=1,
    start_date=eval_generator.end_date + timedelta(days=1),
    ev_generator=ev_generator,
    recomputer=Recomputer(recompute_interval=10, sparse=True),
    sim_class=EvaluationSimulator,
)

ic(validation_generator.end_date + timedelta(days=1))
pass

ic| train_generator.end_date + timedelta(days=1): datetime.datetime(2019, 11, 20, 0, 0)
ic| eval_generator.end_date + timedelta(days=1): datetime.datetime(2019, 11, 28, 0, 0)
ic| validation_generator.end_date + timedelta(days=1): datetime.datetime(2019, 12, 13, 0, 0)


In [3]:
from gymportal.environment import *
from extra import unplug_penalty, ranking_schedule


observation_objects = [
    charging_rates_observation_normalized(),
    percentage_of_magnitude_observation(),
    diff_pilots_charging_rates_observation_normalized(),
    cyclical_minute_observation(),
    cyclical_day_observation(),
    cyclical_month_observation(),
    cyclical_minute_observation_stay(),
    energy_delivered_observation_normalized(),
    num_active_stations_observation_normalized(),
    pilot_signals_observation_normalized(),
    cyclical_minute_observation_arrival(),
    cyclical_day_observation_arrival(),
    cyclical_month_observation_arrival(),
]

reward_objects = [
    # current_constraint_violation(),
    # soft_charging_reward(),
    constraint_charging_reward(),
    unplug_penalty(),
    # pilot_charging_rate_difference_penalty(),
]

In [4]:

train_config = {"observation_objects": observation_objects, "action_object": ranking_schedule(),
                "reward_objects": reward_objects,
                "simgenerator": train_generator,
                "meet_constraints": False}

eval_config = train_config | {'simgenerator': eval_generator}
validation_config = train_config | {'simgenerator': validation_generator}

In [5]:
import wandb

wandb.init(project="ppo_test")

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: tsturm (tsturm-university-kassel). Use `wandb login --relogin` to force relogin
I0000 00:00:1730126440.932237   79065 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


In [7]:
from gymnasium.wrappers import FlattenObservation

env = (
    FlattenObservation(
        SingleAgentSimEnv(config=train_config)
    )
)

In [8]:
from pytorch_lightning import Trainer
from ppo_model import PPO
from pytorch_lightning.loggers import WandbLogger

wandb_logger = WandbLogger(log_model="all")

model = PPO(env)
trainer = Trainer(max_epochs=3, logger=wandb_logger)
res = trainer.fit(model)
res

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/y/git/acn-experiments/.conda/lib/python3.9/site-packages/pytorch_lightning/utilities/parsing.py:44: Attribute 'env' removed from hparams because it cannot be pickled. You can suppress this warning by setting `self.save_hyperparameters(ignore=['env'])`.
/home/y/git/acn-experiments/.conda/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.

  | Name   | Type           | Params | Mode 
--------------------------------------------------
0 | critic | Sequential     | 23.9 K | train
1 | actor  | ActorContinous | 24.3 K | train
--------------------------------------------------
48.3 K    Trainable params
0         Non-trainable params
48.3 K    Total params
0.193     Total est

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.
